In [1]:
!pip install git+https://github.com/business-science/pymodeltime.git

  Cloning https://github.com/business-science/pymodeltime.git to /tmp/pip-req-build-5ybv410y
  Running command git clone --filter=blob:none --quiet https://github.com/business-science/pymodeltime.git /tmp/pip-req-build-5ybv410y
  Resolved https://github.com/business-science/pymodeltime.git to commit dcd6f2c3e3612afbb8caa69182ab0ff976f98c40
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.7/416.7 kB 40.2 MB/s eta 0:00:00
     ━━━━

In [2]:
import pytimetk as tk
from pymodeltime import MLModelWrapper
from pymodeltime import ArimaReg, ProphetReg
from pymodeltime import ModelTimeTable, ModelTimeAccuracy, ModelTimeCalibration, ModelTimeForecast,ModelTimeRefit
from pymodeltime import ArimaReg, ProphetReg
from pymodeltime import MLModelWrapper,H2OAutoMLWrapper
from pymodeltime import plot_modeltime_forecast
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
# Load data
import pandas as pd
import pandas as pd
import pytimetk as tk
from sklearn.model_selection import train_test_split
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

import logging

# Set logging level to WARNING to suppress DEBUG and INFO messages
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)


In [3]:
import pandas as pd

url = 'https://raw.githubusercontent.com/business-science/pymodeltime/main/data/data_macro.csv'
df = pd.read_csv(url)
print(df.head())


         date       GDP        IP       EMP
0  1981-01-01  0.001077 -0.001730  0.005952
1  1981-02-01  0.003043  0.004222  0.006830
2  1981-03-01 -0.000820  0.000000 -0.000408
3  1981-04-01  0.011765  0.018938  0.003284
4  1981-05-01 -0.008364 -0.017627  0.002648


In [4]:

df['date'] = pd.to_datetime(df['date'])
# Get the maximum (last) date in your dataset
max_date = df['date'].max()

# Create a DataFrame with only the last date
last_date_df = pd.DataFrame({'date': [max_date]})

# Extend this DataFrame with 6 future dates


# Extend this DataFrame with 6 future dates
future_dates = last_date_df.future_frame(
    date_column='date',
    length_out= 6,  # Number of future periods to forecast
    freq='M',
    bind_data=False # Do not combine the original data
)

# Append these future dates to your original data
extended_data = pd.concat([df, future_dates]).reset_index(drop=True)

pol_df_dates = extended_data.augment_timeseries_signature(date_column = 'date')
pol_df_dates.head()


,date,GDP,IP,EMP,date_index_num,date_year,date_year_iso,date_yearstart,date_yearend,date_leapyear,...,date_mday,date_qday,date_yday,date_weekend,date_hour,date_minute,date_second,date_msecond,date_nsecond,date_am_pm
0,1981-01-01,0.001077,-0.001730,0.005952,347155200,1981,1981,1,0,0,...,1,1,1,0,0,0,0,0,0,am
1,1981-02-01,0.003043,0.004222,0.006830,349833600,1981,1981,0,0,0,...,1,32,32,1,0,0,0,0,0,am
2,1981-03-01,-0.000820,0.000000,-0.000408,352252800,1981,1981,0,0,0,...,1,60,60,1,0,0,0,0,0,am
3,1981-04-01,0.011765,0.018938,0.003284,354931200,1981,1981,0,0,0,...,1,1,91,0,0,0,0,0,0,am
4,1981-05-01,-0.008364,-0.017627,0.002648,357523200,1981,1981,0,0,0,...,1,31,121,0,0,0,0,0,0,am


In [5]:
pol_df_dates = pol_df_dates[[
    'date',
    'GDP',
    'IP',
    'EMP'
]]

pol_df_dates.tail(10)

,date,GDP,IP,EMP
510,2023-07-01,-0.000741,-0.001397,-0.000317
511,2023-08-01,-0.000497,-0.005717,0.001977
512,2023-09-01,0.000431,-0.001401,0.003152
513,2023-10-01,0.001072,-0.001685,0.000863
514,2023-11-30,NaN,NaN,NaN
515,2023-12-31,NaN,NaN,NaN
516,2024-01-31,NaN,NaN,NaN
517,2024-02-29,NaN,NaN,NaN
518,2024-03-31,NaN,NaN,NaN
519,2024-04-30,NaN,NaN,NaN


In [6]:
columns_to_lag = ['GDP', 'IP', 'EMP']

# Ensure 'date' is in datetime format
pol_df_dates['date'] = pd.to_datetime(pol_df_dates['date'])

# Creating lagged features
df_with_lags = pol_df_dates.augment_lags(
    date_column='date',
    value_column=columns_to_lag,
    lags=[6]
)

# Dropping original IP and EMP columns as you mentioned
df_with_lags = df_with_lags.drop(columns=['IP', 'EMP'])



# Display the first 20 rows of the filtered DataFrame
df_with_lags.head(10)





,date,GDP,GDP_lag_6,IP_lag_6,EMP_lag_6
0,1981-01-01,0.001077,NaN,NaN,NaN
1,1981-02-01,0.003042,NaN,NaN,NaN
2,1981-03-01,-0.000820,NaN,NaN,NaN
3,1981-04-01,0.011765,NaN,NaN,NaN
4,1981-05-01,-0.008362,NaN,NaN,NaN
5,1981-06-01,-0.001484,NaN,NaN,NaN
6,1981-07-01,-0.003620,0.001077,-0.001730,0.005951
7,1981-08-01,-0.008110,0.003042,0.004223,0.006828
8,1981-09-01,0.011238,-0.000820,0.000000,-0.000408
9,1981-10-01,-0.007065,0.011765,0.018936,0.003284


In [7]:
train = df_with_lags[df_with_lags['GDP'].notna()]
train = train.dropna()
train.head()

,date,GDP,GDP_lag_6,IP_lag_6,EMP_lag_6
6,1981-07-01,-0.003620,0.001077,-0.001730,0.005951
7,1981-08-01,-0.008110,0.003042,0.004223,0.006828
8,1981-09-01,0.011238,-0.000820,0.000000,-0.000408
9,1981-10-01,-0.007065,0.011765,0.018936,0.003284
10,1981-11-01,-0.000347,-0.008362,-0.017624,0.002647


In [8]:

future = df_with_lags[df_with_lags.GDP.isnull()]
future.tail()

,date,GDP,GDP_lag_6,IP_lag_6,EMP_lag_6
515,2023-12-31,NaN,-0.000634,0.002136,0.002974
516,2024-01-31,NaN,-0.000741,-0.001396,-0.000317
517,2024-02-29,NaN,-0.000497,-0.005718,0.001976
518,2024-03-31,NaN,0.000431,-0.001401,0.003153
519,2024-04-30,NaN,0.001072,-0.001685,0.000863


In [9]:

train.to_csv("./train.csv",index = False)

import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('./train.csv')

data['date'] = pd.to_datetime(data['date'])

# Assuming 'data' is your initial DataFrame
all_columns = train.columns.tolist()
excluded_columns = ['date', 'GDP']  # Columns to exclude
columns_to_convert = [col for col in all_columns if col not in excluded_columns]

# Convert columns in the original DataFrame
data.loc[:, columns_to_convert] = data.loc[:, columns_to_convert].astype('float32')

# Splitting the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)


test_data.loc[:, columns_to_convert] = test_data[columns_to_convert].astype('float32')

test_data.head(2)

,date,GDP,GDP_lag_6,IP_lag_6,EMP_lag_6
406,2015-05-01,-0.002083,-0.000709,-0.003683,0.000067
407,2015-06-01,0.003698,0.001893,0.003983,0.001878


In [10]:
# Define the target column
target_column = 'GDP'

# Select feature columns - all columns except 'date', 'value', and 'id'
feature_columns = [col for col in train_data.columns if col not in ['date', target_column]]

# Extract features (X) and target (y) from the training data
X_train = train_data[feature_columns]
y_train = train_data[target_column]

# Display the first few rows of X_train to verify
X_test = test_data[feature_columns]
y_test = test_data[target_column]

In [11]:

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Create instances of the ML models
rf_model = RandomForestRegressor()
ml_rf_wrapper = MLModelWrapper(rf_model, feature_columns, "Random Forest_sk")

xgb_model = XGBRegressor()
ml_xgb_wrapper = MLModelWrapper(xgb_model, feature_columns, "XGBoost_sk")

# Fit the models (assuming train_data and y_train are defined)
ml_rf_wrapper.fit(train_data, y_train)
ml_xgb_wrapper.fit(train_data, y_train)

# Make predictions with the Random Forest model
rf_predictions = ml_rf_wrapper.predict(test_data)

# Make predictions with the XGBoost model
xgb_predictions = ml_xgb_wrapper.predict(test_data)


# Example of evaluating the model performance (assuming you have y_test)
from sklearn.metrics import mean_squared_error

# Assuming y_test contains the true target values for the test set

# Calculate Mean Squared Error for Random Forest model
rf_mse = mean_squared_error(y_test, rf_predictions)

# Calculate Mean Squared Error for XGBoost model
xgb_mse = mean_squared_error(y_test, xgb_predictions)

print(f"Random Forest MSE: {rf_mse}")
print(f"XGBoost MSE: {xgb_mse}")


Random Forest MSE: 0.0002493046797023195
XGBoost MSE: 0.00025586258905652034


In [12]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, Lars, LassoLars, OrthogonalMatchingPursuit, BayesianRidge, ARDRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

def tune_model(model, X, y, param_grid, cv=5):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
    grid_search.fit(X, y)
    return grid_search.best_estimator_



ridge_param_grid = {
    'alpha': [0.1, 1, 10]
}


lasso_param_grid = {
    'alpha': [0.1, 1, 10],
    'max_iter': [1000, 5000]
}

mlp_param_grid = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

# Tune and fit Ridge Regression
fine_tuned_ridge = tune_model(Ridge(), X_train, y_train, ridge_param_grid)
ml_ridge_wrapper = MLModelWrapper(fine_tuned_ridge, feature_columns, "Ridge Regression")
ml_ridge_wrapper.fit(X_train, y_train)

# Tune and fit Lasso Regression
fine_tuned_lasso = tune_model(Lasso(), X_train, y_train, lasso_param_grid)
ml_lasso_wrapper = MLModelWrapper(fine_tuned_lasso, feature_columns, "Lasso Regression")
ml_lasso_wrapper.fit(X_train, y_train)

# Tune and fit MLP Regressor
fine_tuned_mlp = tune_model(MLPRegressor(), X_train, y_train, mlp_param_grid)
ml_mlp_wrapper = MLModelWrapper(fine_tuned_mlp, feature_columns, "MLP Regressor")
ml_mlp_wrapper.fit(X_train, y_train)


Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits


In [13]:

import h2o
from h2o.automl import H2OAutoML
from pymodeltime import H2OAutoMLWrapper

# Start H2O cluster
h2o.init()

# Convert your DataFrame to H2OFrame
h2o_train = h2o.H2OFrame(train_data)

# Initialize and train H2O AutoML model
automl = H2OAutoML(max_models= 5, seed=1, max_runtime_secs= 3600)
automl.train(x=feature_columns, y=target_column, training_frame=h2o_train)

# Get the best model
best_h2o_model = automl.leader

# Wrap the model with H2OAutoMLWrapper
h2o_automl_wrapper = H2OAutoMLWrapper(best_h2o_model, target_column=target_column)



Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpn4xodajk
  JVM stdout: /tmp/tmpn4xodajk/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpn4xodajk/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,"14 days, 1 hour and 7 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_loepwu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,30 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,"14 days, 1 hour and 7 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_loepwu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.168 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [14]:
# Extract the leaderboard
lb = automl.leaderboard

# Optionally, you can convert the leaderboard to a pandas DataFrame for easier manipulation
lb_df = lb.as_data_frame()

# Display the leaderboard
print(lb_df.head(10))


Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
                                            model_id      rmse       mse  \
0                     GBM_1_AutoML_1_20240103_173859  0.004272  0.000018   
1  StackedEnsemble_BestOfFamily_1_AutoML_1_202401...  0.004298  0.000018   
2  StackedEnsemble_AllModels_1_AutoML_1_20240103_...  0.004298  0.000018   
3                     GLM_1_AutoML_1_20240103_173859  0.004302  0.000019   
4                     DRF_1_AutoML_1_20240103_173859  0.004650  0.000022   
5                 XGBoost_1_AutoML_1_20240103_173859  0.004746  0.000023   
6                 XGBoost_2_AutoML_1_20240103_173859  0.004931  0.000024   

        mae     rmsle  mean_residual_deviance  
0  0.003198  0.004267                0.000018  
1  0.003227  0.004292                0.000018  
2  0.003226  0.004293                0.000018  
3  0.003213  0.004296                0.000019  
4  0.003530  0.004645                0.000022  
5  0.003

In [15]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from pymodeltime import AutoGluonTabularWrapper

auto_gluon_wrapper = AutoGluonTabularWrapper(target_column="GDP")
auto_gluon_wrapper.fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240103_173922"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20240103_173922"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 

Calibration completed for AutoGluonTabular model.


In [16]:
prophet_model_multi  = ProphetReg(seasonality_yearly=True, seasonality_weekly=True,
                               seasonality_daily=False, changepoint_range=0.5, prior_scale_seasonality=5.0,
                               season='multiplicative',interval_width=0.95 )


prophet_model_multi.fit(train_data,
                        target_column='GDP',
                        date_column='date',
                        regressors= feature_columns)



# Initialize and fit the ARIMA model
arima_reg_model_multi = ArimaReg(
    non_seasonal_ar=1,
    non_seasonal_differences=0,
    non_seasonal_ma=1,
    seasonal_ar=0,
    seasonal_differences=1,
    seasonal_ma=1,
    seasonal_period=12,trend= 't',uses_date=True
) # (no trend), 'c' (constant), 't' (linear trend), or 'ct' (constant and linear trend).


arima_reg_model_multi.fit(train_data,
                          target_column='GDP',
                          date_column='date',
                          regressors= feature_columns)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpwxkdubkf/ijn2gy99.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwxkdubkf/ac__vs3q.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=6390', 'data', 'file=/tmp/tmpwxkdubkf/ijn2gy99.json', 'init=/tmp/tmpwxkdubkf/ac__vs3q.json', 'output', 'file=/tmp/tmpwxkdubkf/prophet_modell46vj478/prophet_model-20240103173934.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:39:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:39:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [17]:


from ipywidgets import interact
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pandas as pd


modeltime_table = ModelTimeTable(ml_rf_wrapper ,ml_xgb_wrapper,
                            ml_ridge_wrapper,ml_lasso_wrapper,ml_mlp_wrapper ,auto_gluon_wrapper,
                                  prophet_model_multi ,arima_reg_model_multi)



model_time_calibrator = ModelTimeCalibration(modeltime_table, test_data,target_column = 'GDP')
model_time_calibrator.calibrate()
calibration_results_df = model_time_calibrator.get_calibration_results()

print(calibration_results_df.to_string(index=False))

Calibration completed for AutoGluonTabular model.
 .model_id   .model             .model_desc    .type  .calibration_data
         1 <fit[+]>        Random Forest_sk ML Model <tibble [102 × 4]>
         2 <fit[+]>              XGBoost_sk ML Model <tibble [102 × 4]>
         3 <fit[+]>        Ridge Regression ML Model <tibble [102 × 4]>
         4 <fit[+]>        Lasso Regression ML Model <tibble [102 × 4]>
         5 <fit[+]>           MLP Regressor ML Model <tibble [102 × 4]>
         6 <fit[+]>     WeightedEnsemble_L2   Custom <tibble [102 × 4]>
         7 <fit[+]>                 PROPHET   Custom <tibble [102 × 4]>
         8 <fit[+]> ARIMA(1,0,1)(0,1,1)[12]   Custom <tibble [102 × 4]>


In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np


modeltime_accuracy = ModelTimeAccuracy(modeltime_table, test_data, target_column='GDP')
accuracy_results = modeltime_accuracy.calculate_accuracy()
print(accuracy_results.to_string(index=False))

 model_id              model_desc      mae     rmse        r2
        1        Random Forest_sk 0.005882 0.015789 -0.034271
        2              XGBoost_sk 0.006273 0.015996 -0.061477
        3        Ridge Regression 0.005343 0.015544 -0.002389
        4        Lasso Regression 0.005326 0.015534 -0.001105
        5           MLP Regressor 0.005385 0.015576 -0.006536
        6     WeightedEnsemble_L2 0.005910 0.015525  0.000120
        7                 PROPHET 0.005750 0.015689 -0.021136
        8 ARIMA(1,0,1)(0,1,1)[12] 0.005929 0.015749 -0.028922


In [19]:
from pymodeltime import ModelTimeForecast

forecast_test = ModelTimeForecast(modeltime_table, new_data=test_data,
                                  actual_data=data,target_column= target_column)
forecast_results_test = forecast_test.forecast()
forecast_results_test.to_csv("forecast_results_test.csv", index=False)



Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: MLModelWrapper and department: All
Processing predictions for model: AutoGluonTabularWrapper and department: All
Processing predictions for model: ProphetReg and department: All
Original columns before processing for Prophet: Index(['date', 'GDP', 'GDP_lag_6', 'IP_lag_6', 'EMP_lag_6'], dtype='object')
Columns after processing for Prophet: Inde

In [25]:
plot_modeltime_forecast(forecast_results_test)

In [21]:

from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd
import h2o
from pymodeltime import AutoGluonTabularWrapper

from pymodeltime import ModelTimeRefit
from h2o.automl import H2OAutoML
# Initialize the refitter
refitter = ModelTimeRefit(modeltime_table, verbose=True, parallel=True, max_workers=4)

# Refit the models with the new data
new_data = pd.concat([train_data, test_data], ignore_index=True)  # Replace with your entire dataset (train + test)


# Convert only the numeric columns to float32
new_data[feature_columns] = new_data[feature_columns].astype('float32')

refitted_models = refitter.refit_models(new_data, target_column=target_column)

# Generate forecasts using the refitted models
forecast_results_refit = refitter.forecast(new_data, target_column=target_column)

# Concatenate all model forecasts into a single DataFrame
all_forecasts = pd.concat([result for result in forecast_results_refit.values() if result is not None], ignore_index=True)

# Save to CSV
all_forecasts.to_csv("./forecast_results_refit.csv", index=False)

No path specified. Models will be saved in: "AutogluonModels/ag-20240103_173939"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20240103_173939"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 

Model successfully refitted: <pymodeltime.MLModelWrapper.MLModelWrapper object at 0x7ab1c6b0a050>
Model successfully refitted: <pymodeltime.MLModelWrapper.MLModelWrapper object at 0x7ab1c6b0aa10>


	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    10520.52 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...


Model successfully refitted: <pymodeltime.MLModelWrapper.MLModelWrapper object at 0x7ab1c69f1990>


	Types of features in original data (raw dtype, special dtypes):
		('datetime', []) : 1 | ['date']
		('float', [])    : 3 | ['GDP_lag_6', 'IP_lag_6', 'EMP_lag_6']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])                : 3 | ['GDP_lag_6', 'IP_lag_6', 'EMP_lag_6']
		('int', ['datetime_as_int']) : 4 | ['date', 'date.year', 'date.month', 'date.dayofweek']
	0.4s = Fit runtime
	4 features in original data used to generate 7 features in processed data.
	Train Data (Processed) Memory Usage: 0.02 MB (0.0% of available memory)
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwxkdubkf/wel40uvs.json
Data preprocessing and feature engineering runtime = 0.61s ...
AutoGluon will gauge predictive performance using evaluation metric: 'root_mean_squared_error'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.
	To change this, specify the eval_metric parameter of Predictor()
Automati

...

Model successfully refitted: <pymodeltime.ArimaReg.ArimaReg object at 0x7ab1bff560e0>


	-0.0125	 = Validation score   (-root_mean_squared_error)
	1.01s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 3586.83s of the 3586.8s of remaining time.
No improvement since epoch 0: early stopping
	-0.0121	 = Validation score   (-root_mean_squared_error)
	0.74s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ... Training model for up to 3586.05s of the 3586.02s of remaining time.
	-0.0121	 = Validation score   (-root_mean_squared_error)
	0.31s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch ... Training model for up to 3585.71s of the 3585.68s of remaining time.
	-0.0119	 = Validation score   (-root_mean_squared_error)
	2.74s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBMLarge ... Training model for up to 3582.93s of the 3582.91s of remaining time.
	-0.0121	 = Validation score   (-root_mean_squared_error)
	0.59s	 = Training   runtime


Calibration completed for AutoGluonTabular model.


	Train Data (Processed) Memory Usage: 0.02 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.17s ...
AutoGluon will gauge predictive performance using evaluation metric: 'root_mean_squared_error'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.2, Train Rows: 406, Val Rows: 102
User-specified model hyperparameters to be fit:
{
	'NN_TORCH': {},
	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
	'CAT': {},
	'XGB': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix':

Model successfully refitted: <pymodeltime.AutoGluonTabularWrapper.AutoGluonTabularWrapper object at 0x7ab1bff555d0>
Error in forecasting with model <pymodeltime.ArimaReg.ArimaReg object at 0x7ab1bff560e0>: array length 508 does not match index length 1016


In [22]:
forecast_future = ModelTimeForecast(refitted_models, actual_data=new_data, future_data=future, target_column=target_column)
forecast_results_future = forecast_future.forecast()
forecast_results_future.to_csv("forecast_results_future_mulit_refitted.csv", index=False)

In [24]:

plot_modeltime_forecast(forecast_results_future)

